In [1]:
import os 
import json

from keras.preprocessing.image import img_to_array, load_img
from keras.utils import get_file
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing import image
from keras.models import Model

import h5py
import numpy as np
import pickle as pk

In [2]:
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

In [3]:
first_check = VGG16(weights='imagenet')
second_check = pk.load(open("C:/Users/User/Desktop/Car Damage Detection/cdd/car_damage_check/classifier.pickle", 'rb'))
third_check = pk.load(open("C:/Users/User/Desktop/Car Damage Detection/cdd/car_damage_check/FRS/classifier.pickle", 'rb'))
fourth_check = pk.load(open("C:/Users/User/Desktop/Car Damage Detection/cdd/car_damage_check/Severity/classifier.pickle", 'rb'))

In [4]:
with open('cat_counter.pk','rb') as f:
    cat_counter = pk.load(f)

In [5]:
cat_counter

Counter({('n04285008', 'sports_car'): 297.92741485964507,
         ('n03770679', 'minivan'): 269.8945387789281,
         ('n03100240', 'convertible'): 197.22693974507274,
         ('n03930630', 'pickup'): 166.5038147289888,
         ('n02814533', 'beach_wagon'): 138.54238876028103,
         ('n03670208', 'limousine'): 103.08318569662515,
         ('n04037443', 'racer'): 101.128612392582,
         ('n03594945', 'jeep'): 67.20719529600319,
         ('n02974003', 'car_wheel'): 42.48469842632767,
         ('n03769881', 'minibus'): 40.416793352225795,
         ('n04461696', 'tow_truck'): 40.359245027502766,
         ('n03459775', 'grille'): 40.175624556228286,
         ('n02930766', 'cab'): 39.998855227429885,
         ('n03796401', 'moving_van'): 28.633751844376093,
         ('n03977966', 'police_van'): 20.11485464120051,
         ('n04065272', 'recreational_vehicle'): 12.45258856046712,
         ('n04252225', 'snowplow'): 12.083932857378386,
         ('n02701002', 'ambulance'): 9.26646127

In [6]:
cat_list = [k for k, v in cat_counter.most_common()[:27]]

In [7]:
cat_list

[('n04285008', 'sports_car'),
 ('n03770679', 'minivan'),
 ('n03100240', 'convertible'),
 ('n03930630', 'pickup'),
 ('n02814533', 'beach_wagon'),
 ('n03670208', 'limousine'),
 ('n04037443', 'racer'),
 ('n03594945', 'jeep'),
 ('n02974003', 'car_wheel'),
 ('n03769881', 'minibus'),
 ('n04461696', 'tow_truck'),
 ('n03459775', 'grille'),
 ('n02930766', 'cab'),
 ('n03796401', 'moving_van'),
 ('n03977966', 'police_van'),
 ('n04065272', 'recreational_vehicle'),
 ('n04252225', 'snowplow'),
 ('n02701002', 'ambulance'),
 ('n04467665', 'trailer_truck'),
 ('n03345487', 'fire_engine'),
 ('n03445924', 'golfcart'),
 ('n03417042', 'garbage_truck'),
 ('n03791053', 'motor_scooter'),
 ('n03776460', 'mobile_home'),
 ('n04517823', 'vacuum'),
 ('n03478589', 'half_track'),
 ('n02704792', 'amphibian')]

# First Check - Car or Not

In [8]:
CLASS_INDEX = None

In [9]:
CLASS_INDEX_PATH = r"C:\Users\User\Desktop\Car Damage Detection\imagenet_class_index.json"

In [10]:
def get_predictions(preds,top=5):
    global CLASS_INDEX
    
    CLASS_INDEX = json.load(open(CLASS_INDEX_PATH))
    
    results = []
    for pred in preds:
        top_indices = pred.argsort()[-top:][::-1]
        result = [tuple(CLASS_INDEX[str(i)]) + (pred[i],) for i in top_indices]
        result.sort(key=lambda x: x[2], reverse=True)
        results.append(result)
    return results

In [16]:
def prepare_img_224(img_path):
    img = load_img(img_path, target_size=(224,224))
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x

In [17]:
def car_categories_check(img_224, model):
    print("Checking if a Car..")
    out = model.predict(img_224)
    top = get_predictions(out, top=5)
    
    for j in top[0]:
        if j[0:2] in cat_list:
            print("Car Check Passed !\n")
            return True
    return False

# Second Check - Damage or Not

In [12]:
def car_damage_check(classifier):
    print("Checking if there is a damage...")
    base_model = first_check
    train_labels = ['Damaged', 'Not Damaged']
    
    model = Model(inputs=base_model.input, outputs=base_model.get_layer('fc1').output)
    image_size = (224, 224)
    
    img = image.load_img(img_path, target_size=image_size)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    feature = model.predict(x)
    flat = feature.flatten()
    flat = np.expand_dims(flat, axis=0)
    preds = classifier.predict(flat)
    predictions = train_labels[preds[0]]
    
    if train_labels[preds[0]] == 'Damaged':
        print("completed - proceeding to location and severity determination\n")
        return True
    return False

# Third Check - Location Assesment

In [14]:
def location_assesment(classifier):
    print("Validating the damage area - Front, Rear or Side")
    base_model = first_check
    train_labels = ['Front Damage', 'Rear Damage', 'Side Damage']
    
    model = Model(inputs=base_model.input, outputs=base_model.get_layer('fc1').output)
    image_size = (224, 224)

    img = image.load_img(img_path, target_size=image_size)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    feature = model.predict(x)
    flat = feature.flatten()
    flat = np.expand_dims(flat, axis=0)
    preds = classifier.predict(flat)
    predictions = train_labels[preds[0]]
    
    print("Your car is damaged at - " + train_labels[preds[0]])
    print("Location assesment complete")
    print('\n')

# Fourth Check - Severity Assesment

In [44]:
def severity_assesment(classifier):
    print("Validating the Severity...")
    base_model = first_check
    train_labels = ['Minor Damage', 'Moderate Damage', 'Severe Damage']
    
    model = Model(inputs=base_model.input, outputs=base_model.get_layer('fc1').output)
    image_size = (224, 224)

    img = image.load_img(img_path, target_size=image_size)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    feature = model.predict(x)
    flat = feature.flatten()
    flat = np.expand_dims(flat, axis=0)
    preds = classifier.predict(flat)
    predictions = train_labels[preds[0]]
    print("Your Car impact is - " + train_labels[preds[0]])
    print("Severity assesment complete")
    print('\n')

# Engine

In [45]:
def engine(img):
    while True:
        try:
            img_224 = prepare_img_224(img)
            g1 = car_categories_check(img_224, first_check)
            
            if g1 is False:
                print("Are you sure this is a picture of your car? Please submit another picture of the damage.")
                print("Hint: Try zooming in/out, using a different angle or different lighting.")
                break
                
            g2 = car_damage_check(second_check)
                
            if g2 is False:
                print("Are you sure that your car is damaged? Please Submit another picture of dammage")
                print("Hint: Try zooming in/out, using a different angle or different lighting.")
                break
            
            x = location_assesment(third_check)
            y = severity_assesment(fourth_check)
            break
            
        except:
            print("Image not accessabile. PLease try again.")
            break

In [46]:
img_path = 'image3.jpg'

In [47]:
engine(img_path)

Checking if a Car..
1/1 [==============================] - 0s 125ms/step
Car Check Passed !

Checking if there is a damage...
1/1 [==============================] - 0s 228ms/step
completed - proceeding to location and severity determination

Validating the damage area - Front, Rear or Side
1/1 [==============================] - 0s 229ms/step
Your car is damaged at - Rear Damage
Location assesment complete


Validating the Severity...
1/1 [==============================] - 0s 231ms/step
Your Car impact is - Moderate Damage
Severity assesment complete


